# Classic Models

Modelos de dados de loja de venda de modelos de veículos (carros antigos, trens, etc) em escala

## Modelo de dados

<img src='./../../docs/imgs/classicmodels.png' />

## Imports

In [2]:
import pandas as pd
from sqlalchemy import create_engine,text
from sqlalchemy.engine.url import URL
import matplotlib.pyplot as plt 
import plotly.express as ex

## Configs

In [3]:
usuario = 'root'
senha = 'secret'

engine = create_engine(URL(
    drivername = 'mysql+pymysql',
    username = usuario,
    password = senha,
    
    host = 'localhost',
    query = {'charset': 'utf8'},
    port = 3306, 
    database='classicmodels'
))

conn = engine.connect()

In [4]:
q = text('show tables')
tables = pd.read_sql(q, conn)

In [5]:
for table in tables.values:
    print(f'Detalhes da tabela: {table[0]}\n')
    q = text(f'show columns from {table[0]}')
    print(pd.read_sql(q, conn))
    print('\n= = = = = = = = = = = = = = = = = = ')

Detalhes da tabela: customers

                     Field           Type Null  Key Default Extra
0           customerNumber            int   NO  PRI    None      
1             customerName    varchar(50)   NO         None      
2          contactLastName    varchar(50)   NO         None      
3         contactFirstName    varchar(50)   NO         None      
4                    phone    varchar(50)   NO         None      
5             addressLine1    varchar(50)   NO         None      
6             addressLine2    varchar(50)  YES         None      
7                     city    varchar(50)   NO         None      
8                    state    varchar(50)  YES         None      
9               postalCode    varchar(15)  YES         None      
10                 country    varchar(50)   NO         None      
11  salesRepEmployeeNumber            int  YES  MUL    None      
12             creditLimit  decimal(10,2)  YES         None      

= = = = = = = = = = = = = = = = = = 
Detalhe

In [6]:
# Consulta de clientes (10) e presentação dos 5  resultados
q = text('select * from customers limit 10')
pd.read_sql(q, conn).head()

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300.0
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200.0
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700.0


In [7]:
# Contagem de clientes por país ordenado pela quantidade, mostrando primeiros 5 resultados
q = text('''
select count(customerNumber) as qty_customers, country 
    from customers 
    group by country 
    order by count(customerNumber) desc
''')
                  
pd.read_sql(q, conn).head()

,qty_customers,country
0,36,USA
1,13,Germany
2,12,France
3,7,Spain
4,5,UK


In [8]:
# Consulta similar à anterior, porém a contagem é tratado na instância do pandas
q = text('select country from customers')
pd.read_sql(q, conn).value_counts().head()

country
USA        36
Germany    13
France     12
Spain       7
UK          5
dtype: int64

In [9]:
# Recuperação das categorias (products lines)
q = text('select * from productlines')
productlines = pd.read_sql(q, conn)
productlines

,productLine,textDescription,htmlDescription,image
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,None,None
1,Motorcycles,Our motorcycles are state of the art replicas ...,None,None
2,Planes,"Unique, diecast airplane and helicopter replic...",None,None
3,Ships,The perfect holiday or anniversary gift for ex...,None,None
4,Trains,Model trains are a rewarding hobby for enthusi...,None,None
5,Trucks and Buses,The Truck and Bus models are realistic replica...,None,None
6,Vintage Cars,Our Vintage Car models realistically portray a...,None,None


In [10]:
# Seleção de quantidade de produtos por categoria ornadenados na camada pandas pelas que tem mais produtos 
q = text('''
select a.productLine as product_line, 
    a.textDescription as description, 
    count(b.productCode) as qty_prd
    from productlines a 
    inner join products b on a.productLine = b.productLine 
    group by a.productLine
''')
productlines = pd.read_sql(q, conn)
productlines.sort_values(by='qty_prd', ascending=False)

,product_line,description,qty_prd
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,38
6,Vintage Cars,Our Vintage Car models realistically portray a...,24
1,Motorcycles,Our motorcycles are state of the art replicas ...,13
2,Planes,"Unique, diecast airplane and helicopter replic...",12
5,Trucks and Buses,The Truck and Bus models are realistic replica...,11
3,Ships,The perfect holiday or anniversary gift for ex...,9
4,Trains,Model trains are a rewarding hobby for enthusi...,3


In [11]:
q = text('''
select productCode as code,  
    productName as name,
    productLine as product_line
    from products
''')

products = pd.read_sql(q, conn)
products.head()

,code,name,product_line
0,S10_1678,1969 Harley Davidson Ultimate Chopper,Motorcycles
1,S10_1949,1952 Alpine Renault 1300,Classic Cars
2,S10_2016,1996 Moto Guzzi 1100i,Motorcycles
3,S10_4698,2003 Harley-Davidson Eagle Drag Bike,Motorcycles
4,S10_4757,1972 Alfa Romeo GTA,Classic Cars


In [12]:
# Consulta de quantidade de produtos por categoria realizada apenas na camada pandas
productlines[['product_line', 'description']].merge(products[['code','product_line']], \
    how='inner', on='product_line')[['product_line', 'code']].groupby('product_line').count()



,code
product_line,
Classic Cars,38
Motorcycles,13
Planes,12
Ships,9
Trains,3
Trucks and Buses,11
Vintage Cars,24


In [13]:
# alternativa melhor
products.groupby('product_line').size().reset_index().rename(columns={0: 'qty_prd'})

,product_line,qty_prd
0,Classic Cars,38
1,Motorcycles,13
2,Planes,12
3,Ships,9
4,Trains,3
5,Trucks and Buses,11
6,Vintage Cars,24


In [14]:
# Seleção de categorias com descrição e quantide de produtos em cada uma delas

result_tmp = productlines[['product_line', 'description']] \
    .merge(products[['code','product_line']], how='inner', on='product_line') \
    [['product_line', 'code']].groupby('product_line').count() \
    .merge(productlines[['product_line', 'description']], how='inner', on='product_line')

# Melhora nos cabeçalhos
result_tmp.columns = ['Product Line', 'Qty Prods', 'Description']
result_tmp

,Product Line,Qty Prods,Description
0,Classic Cars,38,Attention car enthusiasts: Make your wildest c...
1,Motorcycles,13,Our motorcycles are state of the art replicas ...
2,Planes,12,"Unique, diecast airplane and helicopter replic..."
3,Ships,9,The perfect holiday or anniversary gift for ex...
4,Trains,3,Model trains are a rewarding hobby for enthusi...
5,Trucks and Buses,11,The Truck and Bus models are realistic replica...
6,Vintage Cars,24,Our Vintage Car models realistically portray a...


In [15]:
# Alternativa
result_tmp = pd.DataFrame(productlines[['product_line', 'description']] \
    .merge(products[['code','product_line']], how='inner', on='product_line') \
    .groupby(['product_line', 'description'])['product_line'].count())

# Melhora nos cabeçalhos e exclusão dos índices
result_tmp.columns = ['Qty Prods']
result_tmp.reset_index(inplace=True)
result_tmp.columns = ['Description', 'Product Line', 'Qty Prods']

result_tmp

,Description,Product Line,Qty Prods
0,Classic Cars,Attention car enthusiasts: Make your wildest c...,38
1,Motorcycles,Our motorcycles are state of the art replicas ...,13
2,Planes,"Unique, diecast airplane and helicopter replic...",12
3,Ships,The perfect holiday or anniversary gift for ex...,9
4,Trains,Model trains are a rewarding hobby for enthusi...,3
5,Trucks and Buses,The Truck and Bus models are realistic replica...,11
6,Vintage Cars,Our Vintage Car models realistically portray a...,24
